In [1]:
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io
import glob
import re
import os
import shutil

# 设置路径
pyocr.tesseract.TESSERACT_CMD = r'D:\Program Files\ocr\Tesseract-OCR\tesseract.exe'

# 文件夹路径
# path = r"D:\ocr\自建房合法合规性问题整治验收销号单11.3\2"

# 获取配置好的 tesseract 便于后面调用
tool = pyocr.get_available_tools()[0]

In [2]:
base = r"D:\ocr\12.6自建房资料"
list = [];
file_list = os.listdir(base)
for file_name in file_list:
    list.append(file_name)

for i in list:
    print(i)

1
2
3
4
5
6
7
8


## 批量处理

In [3]:
for i in list:
    path = base + r'\{}'.format(i)
    print(path)
    cnt = 0
    
    # 列出目录下的所有文件和文件夹
    file_list = os.listdir(path)
    
    for file_name in file_list:
        # file_extension = os.path.splitext(file_name)[-1].lower()
        # if file_extension != '.pdf':
        #     continue

        # 跳过已经命名的文件
        if len(file_name) == 10:
            continue
        
        image_pdf = Image(filename=path + r'\{}'.format(file_name), resolution=300)
        image_jpeg = image_pdf.convert('jpeg')
        
        # 将图片解析为二进制矩阵
        image_lst = []
        for img in image_jpeg.sequence:
            img_page = Image(image=img)
            image_lst.append(img_page.make_blob('jpeg'))
        # 用 io 模块的 BytesIO 方法读取二进制内容为图片形式，并预览图片
        new_img = PI.open(io.BytesIO(image_lst[0]))
        # new_img.show()
    
        # 解析房屋排查编号
        left = 640
        top = 480
        right = left + 580
        bottom = top + 200
        
        image_obj1 = new_img.crop((left, top, right, bottom))
        # image_obj1.show()
    
        # 截取成功后可以交给 OCR
        txt1 = tool.image_to_string(image_obj1)
        # print(txt1)
        
        req = 'H(.*)'
        tmp = re.findall(req, txt1)
        txt1_real = ''
        if len(tmp):
            txt1_real = ''.join(tmp[0].split())[9:15]
    
        # print(txt1_real)
        if len(txt1_real) != 6 or not txt1_real.isdigit():
            cnt += 1
            print("miss file : {}".format(file_name))
            continue
            
        # 用后 6 位重命名
        newfile = path + r'\{}.pdf'.format(txt1_real)
        cnt = 2
        while os.path.exists(newfile):
            # print("file {} exists.".format(file_name))
            newfile = path + r'\{} ({}).pdf'.format(txt1_real, cnt)
            cnt += 1
            print("重复的文件：{}".format(newfile))
        os.rename(path + r'\{}'.format(file_name), newfile)

    # print('successful!')
    # print("miss {} files !".format(cnt))
    

D:\ocr\12.6自建房资料\1
重复的文件：D:\ocr\12.6自建房资料\1\632299 (2).pdf
重复的文件：D:\ocr\12.6自建房资料\1\667913 (2).pdf
重复的文件：D:\ocr\12.6自建房资料\1\656849 (2).pdf
D:\ocr\12.6自建房资料\2
D:\ocr\12.6自建房资料\3
重复的文件：D:\ocr\12.6自建房资料\3\644191 (2).pdf
D:\ocr\12.6自建房资料\4
D:\ocr\12.6自建房资料\5
重复的文件：D:\ocr\12.6自建房资料\5\542274 (2).pdf
D:\ocr\12.6自建房资料\6
重复的文件：D:\ocr\12.6自建房资料\6\640559 (2).pdf
重复的文件：D:\ocr\12.6自建房资料\6\505795 (2).pdf
重复的文件：D:\ocr\12.6自建房资料\6\504920 (2).pdf
D:\ocr\12.6自建房资料\7
重复的文件：D:\ocr\12.6自建房资料\7\461747 (2).pdf
重复的文件：D:\ocr\12.6自建房资料\7\395484 (2).pdf
重复的文件：D:\ocr\12.6自建房资料\7\399566 (2).pdf
D:\ocr\12.6自建房资料\8
重复的文件：D:\ocr\12.6自建房资料\8\384208 (2).pdf


## 处理单个 pdf

In [16]:
path = r"D:\ocr\12.5晚上自建房\1"
file_name = '202312051434_0038.pdf'
# 以 202309181637_0048.pdf 为例
image_pdf = Image(filename=path + r'\{}'.format(file_name), resolution=300)
image_jpeg = image_pdf.convert('jpeg')

# 将图片解析为二进制矩阵
image_lst = []
for img in image_jpeg.sequence:
    img_page = Image(image=img)
    image_lst.append(img_page.make_blob('jpeg'))

# 用 io 模块的 BytesIO 方法读取二进制内容为图片形式，并预览图片
new_img = PI.open(io.BytesIO(image_lst[0]))
# new_img.show()

# 解析房屋排查编号
# left = 620
# top = 600
# right = left + 580
# bottom = top + 200
left = 640
top = 480
right = left + 580
bottom = top + 200

image_obj1 = new_img.crop((left, top, right, bottom))
image_obj1.show()

# 截取成功后可以交给 OCR
txt1 = tool.image_to_string(image_obj1)
print(txt1)

# req = 'YH(.*)'
req = 'H(.*)'
tmp = re.findall(req, txt1)
txt1_real = ''
if len(tmp):
    txt1_real = ''.join(tmp[0].split())[9:15]
    print("this: " + txt1_real)

I

J| YH430182000462648
this: 462648


In [23]:
# 用后 6 位重命名
if len(txt1_real) == 6 and txt1_real != file_name:
    os.rename(path + r'\{}'.format(file_name), path + r'\{}.pdf'.format(txt1_real))